In [ ]:
from dash import Dash, dash_table, dcc, callback, Output, Input, clientside_callback, _dash_renderer
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html
import webbrowser
from threading import Timer
from dash_iconify import DashIconify


In [ ]:
CSV_FILES = {
    "Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/CPS_Summary.csv",
    "Brand per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/BPCPS_Summary.csv",
    "Product per Brand": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPBPS_Summary.csv",
    "Product per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPCPS_Summary.csv"
}


In [ ]:
theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

In [ ]:
app = Dash()

app.layout = dmc.MantineProvider(
    children=[
        theme_toggle,
        dmc.Container(
        [
            dmc.Title("Market Analysis Dashboard", 
                      style={"size":"h3", "textAlign": "center"}), 

            dmc.RadioGroup(
                [dmc.Radio(i, value=i) for i in CSV_FILES.keys()],
                id="radio-group",
                value="Category",
                size="md",
            ),
            
            dmc.Select(
                id="category-dropdown",
                data=[],
                placeholder="Select a category",
                searchable=True,
                clearable=True,
                # Important for server-side search:
                searchValue="",
                nothingFoundMessage="No matches found",
            ),

            dmc.Space(h=20),

            html.Div(id="time-series-container")
        ], fluid=True,
    )],
)
@callback(
    Output("category-dropdown", "data"),
    Output("time-series-container", "children"),
    Input("radio-group", "value"),
    Input("category-dropdown", "value"),
    Input("category-dropdown", "searchValue")
)
def combined_update(radio_choice, selected_val, search_val):
    # Map radio choice to search column
    col_map = {
        "Brand per Category": "brand",
        "Product per Category": "product",
        "Product per Brand": "product"
    }

    # Handle the cases where search column is brand/product
    if radio_choice in col_map:
        search_col = col_map[radio_choice]
        if search_col == 'brand':
            df = pd.read_csv(
                CSV_FILES[radio_choice],
                usecols=[search_col, "year", "month", "Popularity Score"]
            )

            # Search filtering
            if search_val:
                matching = df[df[search_col].str.contains(search_val, case=False, na=False)][search_col].unique()
            else:
                matching = df[search_col].unique()

            # Limit options for performance
            options = [{"label": val, "value": val} for val in sorted(matching)[:500]]

            # Validate selection
            if selected_val not in matching:
                selected_val = None

            if not selected_val:
                return options, f"Please select a {search_col} to view the time series data."

            # Filter by the selected value
            filtered = df[df[search_col] == selected_val]
        
        else:
            search_val=""
            df = pd.read_csv(
                CSV_FILES[radio_choice],
                usecols=[search_col, "year", "month", "Popularity Score"]
            )
            options = []
            filtered = df.head(5)  # Show top 5 products (highest Popularity Score)


    else:
        # "Category" case
        df = pd.read_csv(CSV_FILES[radio_choice])
        options = [{"label": c, "value": c} for c in sorted(df["category"].unique())]

        if selected_val not in [opt["value"] for opt in options]:
            selected_val = None

        if not selected_val:
            return options, "Please select a category to view the time series data."

        filtered = df[df["category"] == selected_val]

    # Handle empty dataset
    if filtered.empty:
        return options, f"No data available for the selected {radio_choice.lower()}."

    # Prepare graph
    filtered = filtered.copy()
    filtered["date"] = pd.to_datetime(filtered[["year", "month"]].assign(day=1))
    fig = px.line(filtered, x="date", y="Popularity Score", template='plotly_dark',title=f"{radio_choice} - {selected_val}")

    return options, dcc.Graph(figure=fig)

clientside_callback(
    """
    (switchOn) => {
       document.documentElement.setAttribute('data-mantine-color-scheme', switchOn ? 'dark' : 'light');
       return window.dash_clientside.no_update
    }
    """,
    Output("color-scheme-switch", "id"),
    Input("color-scheme-switch", "checked"),
)


def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == "__main__":
    Timer(1, open_browser).start()  # Wait 1 second then open browser
    app.run(debug=True)